In [1]:
import configparser
import os.path
import re
from sys import stderr
import bitarray
import math, pprint


from petsysbuild.petsys import tofpet2b, tofpet2c, fe_power, daqd
from MuHelper.muhelper.util import Utils

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Welcome to JupyROOT 6.30/04


# How to read registers of TOFPET2 ASIC

In [5]:
def ifchanged(AsicConfig, AsicConfigDefault):
    keys = AsicConfig.getKeys()
    result={}
    for key in keys:
        current_setting = AsicConfig.getValue(key)
        default_setting = AsicConfigDefault.getValue(key)
        current_setting_bits = AsicConfig.getBits(key)
        default_setting_bits = AsicConfigDefault.getBits(key)        
        if current_setting!=default_setting:
            result[key] = [current_setting, default_setting, current_setting_bits, default_setting_bits]
    return result

def printChanged(result):
    keys = list(result.keys())
    keys.sort()
    for key in keys:
        print(f"  {key:<20}: Current {result[key][0]:>3} 0b{result[key][2].to01():<10}; Default {result[key][1]:>3} 0b{result[key][3].to01():<10}")



In [6]:

# Open daqd driver
daqd = daqd.Connection()
daqd.initializeSystem()

INFO: active units on ports:  0
INFO: Evaluation kit: FEB/D with GBE connection @ ( 0,  0)
INFO: Setting FEM  power  ON @ (portID, slaveID) = (0,0)
INFO: FEB/D ( 0,  0) has  2 active ASICs: 4, 5


In [7]:

# Get active Asics and Channels
activeAsics = daqd.getActiveAsics()
activeChannels = [ (portID, slaveID, chipID, channelID) for channelID in range(64) for portID, slaveID, chipID in activeAsics ]
# Get Asic configurations
asicsConfig = daqd.getAsicsConfig(forceAccess=True)

# Select the chip and channel to view
VIEW_PortID     = 0
VIEW_SlaveID    = 0
VIEW_ChipID     = 5
VIEW_ChannelID  = 9


cc = asicsConfig[(VIEW_PortID, VIEW_SlaveID, VIEW_ChipID)].channelConfig[VIEW_ChannelID]
gc = asicsConfig[(VIEW_PortID, VIEW_SlaveID, VIEW_ChipID)].globalConfig

# cc = tofpet2c.AsicChannelConfig()
# gc = tofpet2c.AsicGlobalConfig()

gc_default = tofpet2c.AsicGlobalConfig()
cc_default = tofpet2c.AsicChannelConfig()


## View parameters that are different from default

In [7]:
def printChanged(result):
    keys = list(result.keys())
    keys.sort()
    for key in keys:
        print(f"  {key:<20}: Current "+ Utils.color.blue(f"{result[key][0]:>3}") + ", 0b"+Utils.color.blue(f"{result[key][2].to01():<10}") +\
               f"; Default {result[key][1]:>3} 0b{result[key][3].to01():<10}")


In [8]:
gc_default = tofpet2c.AsicGlobalConfig()
gc_default[0:len(gc_default)] = tofpet2c.GlobalConfigAfterReset
diff = ifchanged(gc,gc_default)
# pprint.pprint(diff)
print("Global parameters")
printChanged(diff)

# cc = tofpet2c.AsicChannelConfig()
cc_default = tofpet2c.AsicChannelConfig()
cc_default[0:len(cc_default)] = tofpet2c.ChannelConfigAfterReset
diff = ifchanged(cc,cc_default)
# pprint.pprint(diff)
print("Channel parameters")
printChanged(diff)

Global parameters


NameError: name 'Utils' is not defined

In [8]:
print("------------------Global default----------------------")
gc_default.printAllValues()
print("\n\n------------------Channel default----------------------")
cc_default.printAllValues()

------------------Global default----------------------
                     tx_nlinks :   2 :       bitarray('10') : 1..0
                        tx_ddr :   1 :        bitarray('1') : 2..2
                       tx_mode :   2 :       bitarray('10') : 4..3
                    debug_mode :   0 :        bitarray('0') : 5..5
                     veto_mode :   0 :   bitarray('000000') : 11..6
                   tdc_clk_div :   1 :        bitarray('1') : 12..12
                      r_clk_en :   6 :      bitarray('110') : 15..13
                  stop_ramp_en :   0 :       bitarray('00') : 19..18
                    counter_en :   0 :        bitarray('0') : 20..20
                counter_period :   6 :      bitarray('110') : 23..21
                tac_refresh_en :   1 :        bitarray('1') : 24..24
            tac_refresh_period :   9 :     bitarray('1001') : 28..25
                  data_clk_div :   0 :       bitarray('00') : 30..29
                      unused_1 :   0 :        bitarray('0

In [11]:
cc.getValue("qdc_mode")

1